In [2]:
print("ok")

ok


In [3]:
%pwd

'd:\\genAI_projects\\AI_medical_chatbot\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'd:\\genAI_projects\\AI_medical_chatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data from PDF files
def load_pdf_file(data):
    loader = DirectoryLoader(data, 
                            glob="*.pdf", 
                            loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf_file(data="Data/")

In [9]:
#Split the documents into chunks
def text_splitter(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_splitter(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 6973


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\sanka\AppData\Local\Temp\ipykernel_25148\2604741814.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ai-medical-chatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "ai-medical-chatbot",
    "metric": "cosine",
    "host": "ai-medical-chatbot-jsi1ofb.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [21]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [22]:
# Embed each chunk and upset the embeddings to Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks, 
    index_name=index_name,
    embedding=embeddings, 
)

In [23]:
#Load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch

In [25]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [26]:
retrieved_docs = retriever.invoke("What is a Cesarean section?")

In [27]:
retrieved_docs

[Document(id='5cfe36ae-fee5-4c64-aa86-251819ec822f', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 147.0, 'page_label': '148', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='assisted birth, so forceps may be a better choice if it is\nimportant to remove the baby quickly.\nCesarean sections\nA cesarean section, also called a c-section, is a surgi-\ncal procedure in which incisions are made through a\nwoman’s abdomen and uterus to deliver her baby.\nCesarean sections are performed whenever abnor-\nmal conditions complicate labor and vaginal delivery,\nthreatening the life or health of the mother or the baby.\nThe procedure is performed in the United States on near-'),
 Document(id='fac13d45-bf49-4238-a016-18c3480cd43a', metadata={'author': '', 'creationdate': '2017-05-01T

In [42]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="openai/gpt-5-mini",
    base_url= "https://openrouter.ai/api/v1",
    api_key= "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", 
    temperature=0.4,
    max_tokens=500)

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a helpful AI assistant for question-answering tasks." 
    "Use the following pieces of retrieved context to answer the questions."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "Use three sentences maximum and keep it concise."
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human","{input}"),
])

In [44]:
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [48]:
response = rag_chain.invoke({"input": "What is Dyspepsia?"})
print(response["answer"])

Dyspepsia is painful, difficult, or disturbed digestion. It may be accompanied by other symptoms and can be classified as functional (nonulcer) dyspepsia, sometimes related to abnormal motility of the upper gastrointestinal tract.


In [47]:
response = rag_chain.invoke({"input": "What is a operating system?"})
print(response["answer"])

I don't know — the provided context does not include a definition of "operating system." If you want, I can provide a definition and brief explanation.
